In [1]:
!pip install keras-ocr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 1.2 MB/s 
     |████████████████████████████████| 957 kB 56.5 MB/s 
     |████████████████████████████████| 9.5 MB 48.7 MB/s 
     |████████████████████████████████| 604 kB 65.9 MB/s 
     |████████████████████████████████| 50 kB 7.9 MB/s 
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19582 sha256=3691b3d9d1c8c2f7cb4045f9890cdb24609cbc5fb23bc8ed3cb94a3d6d6a116d
  Stored in directory: /root/.cache/pip/wheels/5f/55/ab/36a76989f7f88d9ca7b1f68da6d94252bb6a8d6ad4f18e04e9
Successfully built validators


In [2]:
import os
import math
import imgaug
#A library for image augmentation in machine learning experiments, 
#particularly convolutional neural networks. Supports the augmentation of images
#, keypoints/landmarks, bounding boxes, heatmaps and 
#segmentation maps in a variety of different ways.
import numpy as np
import matplotlib.pyplot as plt
import sklearn.model_selection
import tensorflow as tf
import keras_ocr
import typing

In [3]:
data_dir = '.'

In [4]:
def get_dataset(training_gt_dir,training_images_dir,skip_illegible=False):
  dataset=[]
  image_files_name = os.listdir(training_images_dir)
  label_files_name = os.listdir(training_gt_dir)
  for image_file, label_file in zip(image_files_name,label_files_name):
        image_path = os.path.join(training_images_dir, image_file )
        lines = []
        with open(os.path.join(training_gt_dir, label_file), "r", encoding="utf8") as f:
            current_line: typing.List[typing.Tuple[np.ndarray, str]] = []
            for raw_row in f.read().split("\n"):
                if raw_row == "":
                    lines.append(current_line)
                    current_line = []
                else:
                    row = raw_row.split(" ")[4:]
                    #take from the 4th element onwards
                    character = row[-1][1:-1]
                    #the last element is the letter
                    if character == "" and skip_illegible:
                        continue
                    x1, y1, x2, y2 = map(float, row[:4])
                    #get 4 final coordinates of file label execpt last letter
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                    #cast type to int
                    current_line.append(
                        (np.array([[x1, y1], [x2, y1], [x2, y2], [x1, y2]]), character)
                    )
        lines = [line for line in lines if line]
        dataset.append((image_path, lines, 1))
  return dataset

In [5]:
training_gt_dir="/content/drive/MyDrive/OCR/Keras/dataset/labels"
training_images_dir="/content/drive/MyDrive/OCR/Keras/dataset/image"

In [9]:
dataset=get_dataset(training_gt_dir,training_images_dir,skip_illegible=False)

In [10]:
train, validation = sklearn.model_selection.train_test_split(
    dataset, train_size=0.8, random_state=42
)
augmenter = imgaug.augmenters.Sequential([
    imgaug.augmenters.Affine(
      scale=(1.0, 1.2),
      rotate=(-5, 5)
      # Apply affine transformations to each image.
      # Scale/zoom them, translate/move them, rotate them and shear them.
    ),
    imgaug.augmenters.GaussianBlur(sigma=(0, 0.5)),
    #Strengthen or weaken the contrast in each image.
    imgaug.augmenters.Multiply((0.8, 1.2), per_channel=0.2)
    # Make some images brighter and some darker.
    # In 20% of all cases, we sample the multiplier once per channel,
    # which can end up changing the color of the images.
])
generator_kwargs = {'width': 640, 'height': 640}
training_image_generator = keras_ocr.datasets.get_detector_image_generator(
    labels=train,
    augmenter=augmenter,
    **generator_kwargs
)
validation_image_generator = keras_ocr.datasets.get_detector_image_generator(
    labels=validation,
    **generator_kwargs
)

In [11]:
detector = keras_ocr.detection.Detector()

batch_size = 1
training_generator, validation_generator = [
    detector.get_batch_generator(
        image_generator=image_generator, batch_size=batch_size
    ) for image_generator in
    [training_image_generator, validation_image_generator]
]
detector.model.fit_generator(
    generator=training_generator,
    steps_per_epoch=math.ceil(len(train) / batch_size),
    epochs=1000,
    workers=0,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(restore_best_weights=True, patience=5),
        #patince: number of epochs with no improvement after which training will be stopped
        #the loss for 5 consecutive epochs.
        tf.keras.callbacks.CSVLogger(os.path.join(data_dir, 'detector_keras.csv')),
        #Callback that streams epoch results to a CSV file.
        tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(data_dir, '/content/drive/MyDrive/detector_keras_vinAI.h5'))
        #Callback to save the Keras model or model weights at some frequency.
    ],

    validation_data=validation_generator,
    validation_steps=math.ceil(len(validation) / batch_size)
)
#val_loss: quantity to be monitored.

Looking for /root/.keras-ocr/craft_mlt_25k.h5


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/1000
960/960 [==============================] - ETA: 0s - loss: 0.0026

/usr/local/lib/python3.7/dist-packages/keras_ocr/tools.py:580: RuntimeWarning: invalid value encountered in double_scalars
  rotation = np.arctan((tl[0] - bl[0]) / (tl[1] - bl[1]))
/usr/local/lib/python3.7/dist-packages/keras_ocr/tools.py:580: RuntimeWarning: invalid value encountered in long_scalars
  rotation = np.arctan((tl[0] - bl[0]) / (tl[1] - bl[1]))


960/960 [==============================] - 669s 679ms/step - loss: 0.0026 - val_loss: 0.4066
Epoch 2/1000
960/960 [==============================] - 378s 394ms/step - loss: 0.0030 - val_loss: 0.4034
Epoch 3/1000
960/960 [==============================] - 379s 395ms/step - loss: 0.0039 - val_loss: 0.4042
Epoch 4/1000
960/960 [==============================] - 380s 396ms/step - loss: 0.0026 - val_loss: 0.4084
Epoch 5/1000
960/960 [==============================] - 377s 392ms/step - loss: 0.0035 - val_loss: 0.4030
Epoch 6/1000
960/960 [==============================] - 377s 393ms/step - loss: 0.0028 - val_loss: 0.3994
Epoch 7/1000
960/960 [==============================] - 376s 392ms/step - loss: 0.0026 - val_loss: 0.4073
Epoch 8/1000
960/960 [==============================] - 376s 392ms/step - loss: 0.0032 - val_loss: 0.4075
Epoch 9/1000
960/960 [==============================] - 376s 391ms/step - loss: 0.0040 - val_loss: 0.3998
Epoch 10/1000
960/960 [==============================] - 37